In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  1


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'sent140'
params['dataset'] = 'mnist'
params['model'] = 'mlp'
#params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['RAC'] = False
  params['RCC'] = False
  params['allow_empty'] = True
  
  # TEST Hyperparameters, Experimental Only
  params['evenly'] = False
  params['min_clients'] = 2
    
  # We implement IFCA and FeSEM base on grouprox
  params['ifca'] = True
  params['fesem'] = False
    
"""
Common parameters
"""
params['num_rounds'] = 300
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ################ Important ######################
params['seed'] = 233
params['agg_lr'] = 0.05

"""
We immplement IFCA and FeSEM base on grouprox,
Some configurations should be flush for above two CFL frameworks
"""
if params['ifca'] == True or params['fesem'] == True:
    params['proximal'] = False
    params['RCC'] = False
    params['allow_empty'] = True
    params['agg_lr'] = 0.0 # IFCA and FeSEM don't use this parameter

""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
    
  if params['ifca'] == True:
    info2 = info2 + '-IFCA'
  if params['fesem'] == True:
    info2 = info2 + '-FeSEM'
  
  info2 = info2 +  '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-mnist-mlp-IFCA-NumG3-Agglr0.0-ProximalF-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
params['sklearn_seed'] = params['seed']
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train


/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
1000 Clients in Total
101770 Parameters in Total
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


In [4]:
"""
Test Ternary Similarity
"""
#x, y = server.test_ternary_cosine_similariy()

'\nTest Ternary Similarity\n'

In [5]:
"""
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)
plt.show()
"""

'\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(7,7))\nplt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)\nplt.show()\n'

In [ ]:
server.train()

Training with 20 workers ---


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Group 0, clients ['f_00298', 'f_00027', 'f_00558']
Group 1, clients ['f_00993', 'f_00553', 'f_00672', 'f_00231']
Group 2, clients ['f_00859', 'f_00971', 'f_00306', 'f_00706', 'f_00496', 'f_00784', 'f_00239', 'f_00578', 'f_00055', 'f_00906', 'f_00175', 'f_00014', 'f_00077']
Group 0
At round 0 accuracy: 0.2857142857142857
At round 0 training accuracy: 0.26229508196721313
At round 0 training loss: 1.9290181593816789
Group 1
At round 0 accuracy: 0.13333333333333333
At round 0 training accuracy: 0.20754716981132076
At round 0 training loss: 2.3486012130413414
Group 2
At round 0 accuracy: 0.09178743961352658
At round 0 training accuracy: 0.10776102735901731
At round 0 training loss: 2.547757662121375
At round 0 mean test accuracy: 0.116 mean train accuracy: 0.13031293787949555 # of test client: 20
The client-group discrepancy are: [21.86091334 21.81561974 21.94770632 21.84466018]
Begin group  0 training


Epoch:  75%|████████████████████████████████████████████████████████▎                  | 15/20 [00:00<00:00, 148.91it/s]

Begin group  1 training


...

Group 0, clients ['f_00786', 'f_00217', 'f_00098', 'f_00446']
Group 1, clients ['f_00930', 'f_00803', 'f_00810', 'f_00983', 'f_00620', 'f_00293']
Group 2, clients ['f_00499', 'f_00069', 'f_00078', 'f_00775', 'f_00358', 'f_00924', 'f_00869', 'f_00168', 'f_00575', 'f_00444']


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Group 0
At round 17 accuracy: 0.9475920679886686
At round 17 training accuracy: 0.9561625358286967
At round 17 training loss: 0.1711770058063241
Group 1
At round 17 accuracy: 0.9516806722689075
At round 17 training accuracy: 0.965331664580726
At round 17 training loss: 0.11962496439621227
Group 2
At round 17 accuracy: 0.9155446756425949
At round 17 training accuracy: 0.9248839907192575
At round 17 training loss: 0.33179778352421935
At round 17 mean test accuracy: 0.9385858585858586 mean train accuracy: 0.9493202670893981 # of test client: 311
The client-group discrepancy are: [17.05674864 11.99154065 15.35553138 20.10356219]
Begin group  0 training


Epoch:  25%|███████████████████▎                                                         | 5/20 [00:00<00:00, 41.83it/s]

Begin group  1 training


In [ ]:
"""
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')
"""